In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP

/content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP


In [1]:
!ls

cached_lm_GPT2TokenizerFast_128_test_dataset.txt
cached_lm_GPT2TokenizerFast_128_test_dataset.txt.lock
cached_lm_GPT2TokenizerFast_128_train_dataset.txt
cached_lm_GPT2TokenizerFast_128_train_dataset.txt.lock
generate_dataset.py
gpt2
layer1.json
LICENSE
Preprocessing.ipynb
README.md
Recipe
RecipeGenerator
RecipeGenerator.xcodeproj
RecipeTraining.ipynb
test_dataset.txt
test.py
train_dataset.txt
Training.py


In [1]:
import re
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [2]:
import json
with open('layer1.json') as json_file:
    data = json.load(json_file)

train_data, test_data = train_test_split(data, test_size = 0.2, shuffle = True)

In [3]:
recipes = data
recipes[0]['title']

'Worlds Best Mac and Cheese'

In [24]:
def get_ingredients(recipe):    
    ingredients = recipe['ingredients']
    measurements = ['oz', 'ounces','ounce', 'cups', 'cup', 'teaspoon', '/', 'to', 'lbs','lb', 'tsp', 'jar']
    item_list = []
    for item in ingredients:
        item = item['text']
        item = item.lower()
    #     item = item.split('(')[0].strip() # remove items in brackets
        item = re.sub(r'\s*\([^)]*\)', '', item)
        item = ''.join([char for char in item if not char.isdigit()]) # remove numbers
        for measurement in measurements:
            item = item.replace(measurement, '').strip() # remove measurements
        item.strip()
        if item:
            item_list.append(item)
    ingredient_list = '<SOI> ' + ' <ISEP> '.join(item_list) + ' <EOI>'
    return ingredient_list

In [25]:
get_ingredients(recipes[1000])

'<SOI> packages crescent rolls <ISEP> sausage, fried and crumbled <ISEP> swiss cheese <ISEP> cheddar cheese <ISEP> eggs <ISEP> oregano <ISEP> garlic powder <ISEP> parmesan cheese,  taste <EOI>'

In [26]:
def get_instructions(recipe):
    instructions = recipe['instructions']
    instr_str = "<SOR> "
    for instr in instructions:
      recipe = re.sub(r"\s", " ", instr['text'])
      instr_str += recipe.lower().strip()
    instr_str += ' <EOR>'
    return instr_str

get_instructions(recipes[32])

'<SOR> for the cupcakes: preheat the oven to 350 degrees f. fill the cupcake pan with paper liners.beat the granulated sugar and butter until light and fluffy in a standing mixer.beat in the eggs.add the lemon zest and juice and beat until combined.sift the flour, baking powder, baking soda and salt in a separate bowl.alternately add the buttermilk and flour mixture to the standing mixer, beating well between each addition.fill the cupcake liners two-thirds full.bake the cupcakes until a toothpick inserted comes out clean, 18 to 22 minutes.refrigerate the cupcakes until they are completely cooled.for the filling: cook 1/3 cup water, 1 cup of the blueberries and granulated sugar over medium heat for 10 minutes, stirring frequently.once the blueberries begin to burst, add the remaining 1 cup blueberries and cook on medium heat, 8 to 10 minutes.the filling is done when it coats the back of a metal spoon.for the frosting: beat the butter and cream cheese until light and fluffy in a standin

In [27]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts['Instructions']).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [28]:
train_path = 'train_dataset2.txt'
test_path = 'test_dataset2.txt'

In [29]:
def build_text_files(recipes, dest_path):
  f = open(dest_path, 'w')
  in_recipe = ''
  for recipe in recipes:
    ingredients = get_ingredients(recipe)
    instr = get_instructions(recipe)
    text = ingredients+" "+instr
    text = re.sub(r"\s", " ", text)
    in_recipe += text
  f.write(in_recipe)


In [ ]:
build_text_files(train_data, train_path)
build_text_files(test_data, test_path)